In [29]:
import os
import itertools
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
from scipy import sparse
from scipy.stats import ttest_ind
from sklearn.datasets import dump_svmlight_file
from sklearn.feature_selection import (
    SelectKBest,
    f_classif,
    mutual_info_classif,
    SelectFromModel,
    RFE
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV, LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score, confusion_matrix, precision_recall_curve
# from imblearn.over_sampling import SMOTE
import xgboost as xgb


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
def load_data(npz_paths: str, data_type: str, threshold=0.5, center=0):
    upper_threshold = threshold + center
    lower_threshold = -threshold + center
    if data_type not in ["X_mean", "X_cls"]:
        raise Exception("data type in valid")

    X_list = []
    y_list = []

    for npz_path in npz_paths:
    
        base = os.path.splitext(os.path.basename(npz_path))[0]      
        csv_path = os.path.join(
            os.path.dirname(npz_path),
            base + "_meta.csv"                     
        )
        csv_path2 = os.path.join(
            os.path.dirname(npz_path),
            # base + "_meta.csv"    
            "_".join(base.split("_")[:4]) + "_features_meta.csv"                                 
        )



        data = np.load(npz_path, allow_pickle=True)
        X = data[data_type]      # (N_docs, 2*D)
        # X_concat = data["X_mean"]
        tids = data["transcriptids"]    


        meta = pd.read_csv(csv_path)
        meta2 = pd.read_csv(csv_path2)

        filter_tids = set(meta2.transcriptid)
        meta = meta[meta.transcriptid.isin(filter_tids)]

        meta_unique = (
            meta[["transcriptid", "SUESCORE", "label"]]
            .drop_duplicates(subset="transcriptid", keep="first")
            .set_index("transcriptid")
        )

        mask_ids = np.isin(tids, meta_unique.index)
        X_filt = X[mask_ids]
        tids_filt = np.array(tids)[mask_ids]


        lab_df = meta.assign(
            label=lambda df: df.SUESCORE.map(
                lambda s: 1 if s >= upper_threshold else (0 if s <= lower_threshold else np.nan)
            )
        )

        # meta['label'] = np.nan

        # set label = 1 where SUESCORE > threshold
        meta.loc[meta['SUESCORE'] >= upper_threshold, 'label'] = 1

        # set label = 0 where SUESCORE < -threshold
        meta.loc[meta['SUESCORE'] <= lower_threshold, 'label'] = 0

        mask_label = lab_df.label.notna().values
        # apply the same mask in the same order as the CSV, so we use .loc on lab_df
        # but first filter lab_df to only those transcriptids in tids_filt
        Xc, y = X_filt[mask_label], meta.loc[mask_label, "label"].astype(int).values
        
        # now align X and y
        # X_final = X_filt[lab_sub.label.notna()]
        # y_final = lab_sub.label.astype(int).values

        # collect
        X_list.append(Xc)
        y_list.append(y)

    # 2. concatenate all files together
    Xc = np.vstack(X_list)   # shape: (sum_i N_i, 2*D)
    y  = np.concatenate(y_list)  # shape: (sum_i N_i,)

    print("Combined Xc shape:", Xc.shape)
    print("Combined y shape: ", y.shape)

    return Xc, y

In [4]:
def downsample_balance(Xc_unbalanced, y_unbalanced):
    # forced resampling
    idx0 = np.where(y_unbalanced == 0)[0]
    idx1 = np.where(y_unbalanced == 1)[0]

    n = min(len(idx0), len(idx1))

    sel0 = np.random.choice(idx0, size=n, replace=False)
    sel1 = np.random.choice(idx1, size=n, replace=False)

    sel = np.concatenate([sel0, sel1])
    np.random.shuffle(sel)

    # slice out your balanced subset
    Xc_out = Xc_unbalanced[sel]
    y_out = y_unbalanced[sel]

    print("Balanced X shape:", Xc_out.shape)
    print("Balanced y counts:", np.bincount(y_out))
    return Xc_out, y_out

In [25]:
train_npz_paths = [
    "./data/doc_features/gemma_2b_cls_filtered/transcript_componenttext_2012_1_cls_mean.npz",    
    "./data/doc_features/gemma_2b_cls_filtered/transcript_componenttext_2012_2_cls_mean.npz",   
    "./data/doc_features/gemma_2b_cls_filtered/transcript_componenttext_2013_1_cls_mean.npz",   
    "./data/doc_features/gemma_2b_cls_filtered/transcript_componenttext_2013_2_cls_mean.npz",   
]

val_npz_paths = [
    "./data/doc_features/gemma_2b_cls_filtered/transcript_componenttext_2014_1_cls_mean.npz",   
    "./data/doc_features/gemma_2b_cls_filtered/transcript_componenttext_2014_2_cls_mean.npz", 
]

gemma_train_embs, y_train = load_data(train_npz_paths, "X_mean", threshold=0.5)
gemma_val_embs, y_val = load_data(val_npz_paths, "X_mean", threshold=0.5)

# Xc, y = load_data(train_npz_paths, "X_concat", threshold=0.1)
# X_val_all_feat, y_val = load_data(val_npz_paths, "X_concat", threshold=0.1)
# X_test_all_feat, y_test = load_data(test_npz_paths, "X_mean")
# Split X_val_all_feat, y_val into two equal parts:
gemma_val_embs, gemma_test_embs, y_val, y_test = train_test_split(
    gemma_val_embs, 
    y_val, 
    test_size=0.5,       # puts half into X_test/y_test
    random_state=42,     # for reproducibility
    # stratify=y_val       # if you want to preserve class proportions
)


Combined Xc shape: (4821, 2304)
Combined y shape:  (4821,)
Combined Xc shape: (2508, 2304)
Combined y shape:  (2508,)


In [26]:
train_npz_paths = [
    "./data/doc_features/llama3_3b_cls_filtered/transcript_componenttext_2012_1_cls_mean.npz",    
    "./data/doc_features/llama3_3b_cls_filtered/transcript_componenttext_2012_2_cls_mean.npz",   
    "./data/doc_features/llama3_3b_cls_filtered/transcript_componenttext_2013_1_cls_mean.npz",   
    "./data/doc_features/llama3_3b_cls_filtered/transcript_componenttext_2013_2_cls_mean.npz",   
]

val_npz_paths = [
    "./data/doc_features/llama3_3b_cls_filtered/transcript_componenttext_2014_1_cls_mean.npz",   
    "./data/doc_features/llama3_3b_cls_filtered/transcript_componenttext_2014_2_cls_mean.npz", 
]

llama_train_embs, y_train = load_data(train_npz_paths, "X_mean", threshold=0.5)
llama_val_embs, y_val = load_data(val_npz_paths, "X_mean", threshold=0.5)

# Xc, y = load_data(train_npz_paths, "X_concat", threshold=0.1)
# X_val_all_feat, y_val = load_data(val_npz_paths, "X_concat", threshold=0.1)
# X_test_all_feat, y_test = load_data(test_npz_paths, "X_mean")
# Split X_val_all_feat, y_val into two equal parts:
llama_val_embs, llama_test_embs, y_val, y_test = train_test_split(
    llama_val_embs, 
    y_val, 
    test_size=0.5,       # puts half into X_test/y_test
    random_state=42,     # for reproducibility
    # stratify=y_val       # if you want to preserve class proportions
)

Combined Xc shape: (4821, 3072)
Combined y shape:  (4821,)
Combined Xc shape: (2508, 3072)
Combined y shape:  (2508,)


In [27]:
train_npz_paths = [
    "./data/doc_features/qwen_4b_cls_filtered/transcript_componenttext_2012_1_cls_mean.npz",    
    "./data/doc_features/qwen_4b_cls_filtered/transcript_componenttext_2012_2_cls_mean.npz",   
    "./data/doc_features/qwen_4b_cls_filtered/transcript_componenttext_2013_1_cls_mean.npz",   
    "./data/doc_features/qwen_4b_cls_filtered/transcript_componenttext_2013_2_cls_mean.npz",   
]

val_npz_paths = [
    "./data/doc_features/qwen_4b_cls_filtered/transcript_componenttext_2014_1_cls_mean.npz",   
    "./data/doc_features/qwen_4b_cls_filtered/transcript_componenttext_2014_2_cls_mean.npz", 
]

qwen_train_embs, y_train = load_data(train_npz_paths, "X_mean", threshold=0.5)
qwen_val_embs, y_val = load_data(val_npz_paths, "X_mean", threshold=0.5)

# Xc, y = load_data(train_npz_paths, "X_concat", threshold=0.1)
# X_val_all_feat, y_val = load_data(val_npz_paths, "X_concat", threshold=0.1)
# X_test_all_feat, y_test = load_data(test_npz_paths, "X_mean")
# Split X_val_all_feat, y_val into two equal parts:
qwen_val_embs, qwen_test_embs, y_val, y_test = train_test_split(
    qwen_val_embs, 
    y_val, 
    test_size=0.5,       # puts half into X_test/y_test
    random_state=42,     # for reproducibility
    # stratify=y_val       # if you want to preserve class proportions
)

Combined Xc shape: (4821, 2560)
Combined y shape:  (4821,)
Combined Xc shape: (2508, 2560)
Combined y shape:  (2508,)


In [30]:
def fit_linear_transformation(X_train, target_train, X_test, target_test, X_val=None, use_skl=True, reg=10.0):
    # train T: llama -> Gemma
    scaler_X = StandardScaler().fit(X_train)
    scaler_Y = StandardScaler().fit(target_train)

    X_tr = scaler_X.transform(X_train)
    Y_tr = scaler_Y.transform(target_train)
    X_te = scaler_X.transform(X_test)
    Y_te = scaler_Y.transform(target_test)

    if X_val is not None:
        X_va = scaler_X.transform(X_val)
        
    alpha = reg
    d_X = X_tr.shape[1]

    if not use_skl:
        I = np.eye(d_X)
        # compute W_closed:
        W_closed = np.linalg.inv(X_tr.T @ X_tr + alpha * I) @ X_tr.T @ Y_tr

        # Map test embeddings and inverse-transform:
        Y_tr_from_transform_scaled = X_tr @ W_closed
        
        if X_val is not None:
            Y_val_from_transform_scaled = X_va @ W_closed

        Y_pred_scaled = X_te @ W_closed
        Y_pred = scaler_Y.inverse_transform(Y_pred_scaled)

        # Evaluate (e.g. MSE or cosine similarity)
        mse = np.mean((Y_pred - scaler_Y.inverse_transform(Y_te))**2)
        print(f"Closed-form ridge MSE: {mse:.4f}")
        print(np.diag(cosine_similarity(Y_pred_scaled, Y_te)).mean())

        if X_val is not None:
            return W_closed, Y_tr_from_transform_scaled, Y_val_from_transform_scaled, Y_pred_scaled
        else:
            return W_closed, Y_tr_from_transform_scaled, Y_pred_scaled
        
    else:
        # 3b) Using sklearn.Ridge:
        # Note: sklearn’s Ridge solves for each output dimension jointly when Y is 2D.
        model = Ridge(alpha=alpha, fit_intercept=False, solver="auto")# intercept is already handled by StandardScaler
        model.fit(X_tr, Y_tr)
        W_sklearn = model.coef_.T   

        Y_tr_from_transform_scaled = model.predict(X_tr)

        if X_val is not None:
            Y_val_from_transform_scaled = model.predict(X_va)
        
        Y_pred_scaled = model.predict(X_te)

        Y_pred = scaler_Y.inverse_transform(Y_pred_scaled)
        mse = np.mean((Y_pred - scaler_Y.inverse_transform(Y_te))**2)
        print(f"sklearn.Ridge MSE: {mse:.4f}")
        print(np.diag(cosine_similarity(Y_pred_scaled, Y_te)).mean())

        if X_val is not None:
            return model, Y_tr_from_transform_scaled, Y_val_from_transform_scaled, Y_pred_scaled
        else:
            return model, Y_tr_from_transform_scaled, Y_pred_scaled

In [32]:
trans_llama_gemma, llama_train_aligned, llama_val_aligned, llama_test_aligned = fit_linear_transformation(llama_train_embs, gemma_train_embs, llama_test_embs, gemma_test_embs, llama_val_embs, use_skl=True, reg=2)
trans_qwen_gemma, qwen_train_aligned, qwen_val_aligned, qwen_test_aligned = fit_linear_transformation(qwen_train_embs, gemma_train_embs, qwen_test_embs, gemma_test_embs, qwen_val_embs, use_skl=False, reg=1)

/home/airlay88/emb-transfer/venv/lib/python3.12/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=5.72943e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


sklearn.Ridge MSE: 0.1413
0.87713283
Closed-form ridge MSE: 0.1618
0.8529351786157778


In [33]:
scaler = StandardScaler().fit(gemma_train_embs)
gemma_train_aligned = scaler.transform(gemma_train_embs)
gemma_val_aligned = scaler.transform(gemma_val_embs)
gemma_test_aligned = scaler.transform(gemma_test_embs)

In [34]:
train_aligned_embs = np.array([gemma_train_aligned, llama_train_aligned, qwen_train_aligned]).mean(axis=0)
val_aligned_embs = np.array([gemma_val_aligned, llama_val_aligned, qwen_val_aligned]).mean(axis=0)
test_aligned_embs = np.array([gemma_test_aligned, llama_test_aligned, qwen_test_aligned]).mean(axis=0)

In [41]:
# X_mean, l2 0.005
param_grid = {"logisticregression__C": [0.0001, 0.005, 0.1]}

pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty="l2",
        solver="saga",    
        # solver="liblinear",    
        # class_weight="balanced",
        max_iter=3000,
        random_state=42
    )
)

search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)
search.fit(gemma_train_aligned, y_train)

print("Best C (inverse reg. strength):", search.best_params_["logisticregression__C"])
print("CV ROC AUC:", search.best_score_)


best_clf = search.best_estimator_
y_pred_probs = best_clf.predict_proba(gemma_test_aligned)[:, 1]
y_pred       = best_clf.predict(gemma_test_aligned)

print(classification_report(y_test, y_pred, digits=4))
print("Test ROC AUC:", roc_auc_score(y_test, y_pred_probs))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best C (inverse reg. strength): 0.005
CV ROC AUC: 0.655509848358663
              precision    recall  f1-score   support

           0     0.3261    0.1807    0.2326       249
           1     0.8172    0.9075    0.8600      1005

    accuracy                         0.7632      1254
   macro avg     0.5716    0.5441    0.5463      1254
weighted avg     0.7197    0.7632    0.7354      1254

Test ROC AUC: 0.6285839876920618


In [42]:
# X_mean, l2 0.005
param_grid = {"logisticregression__C": [0.0001, 0.005, 0.1]}

pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty="l2",
        solver="saga",    
        # solver="liblinear",    
        # class_weight="balanced",
        max_iter=3000,
        random_state=42
    )
)

search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)
search.fit(llama_train_aligned, y_train)

print("Best C (inverse reg. strength):", search.best_params_["logisticregression__C"])
print("CV ROC AUC:", search.best_score_)


best_clf = search.best_estimator_
y_pred_probs = best_clf.predict_proba(llama_test_aligned)[:, 1]
y_pred       = best_clf.predict(llama_test_aligned)

print(classification_report(y_test, y_pred, digits=4))
print("Test ROC AUC:", roc_auc_score(y_test, y_pred_probs))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best C (inverse reg. strength): 0.005
CV ROC AUC: 0.6575908618991767
              precision    recall  f1-score   support

           0     0.2945    0.1727    0.2177       249
           1     0.8141    0.8975    0.8538      1005

    accuracy                         0.7536      1254
   macro avg     0.5543    0.5351    0.5357      1254
weighted avg     0.7109    0.7536    0.7275      1254

Test ROC AUC: 0.6278966612719535


In [43]:
# X_mean, l2 0.005
param_grid = {"logisticregression__C": [0.0001, 0.005, 0.1]}

pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty="l2",
        solver="saga",    
        # solver="liblinear",    
        # class_weight="balanced",
        max_iter=3000,
        random_state=42
    )
)

search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)
search.fit(qwen_train_aligned, y_train)

print("Best C (inverse reg. strength):", search.best_params_["logisticregression__C"])
print("CV ROC AUC:", search.best_score_)


best_clf = search.best_estimator_
y_pred_probs = best_clf.predict_proba(qwen_test_aligned)[:, 1]
y_pred       = best_clf.predict(qwen_test_aligned)

print(classification_report(y_test, y_pred, digits=4))
print("Test ROC AUC:", roc_auc_score(y_test, y_pred_probs))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best C (inverse reg. strength): 0.005
CV ROC AUC: 0.6580540221129143
              precision    recall  f1-score   support

           0     0.2886    0.1727    0.2161       249
           1     0.8136    0.8945    0.8521      1005

    accuracy                         0.7512      1254
   macro avg     0.5511    0.5336    0.5341      1254
weighted avg     0.7093    0.7512    0.7258      1254

Test ROC AUC: 0.6194209674518971


In [44]:
# X_mean, l2 0.005
param_grid = {"logisticregression__C": [0.0001, 0.005, 0.1]}

pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty="l2",
        solver="saga",    
        # solver="liblinear",    
        # class_weight="balanced",
        max_iter=3000,
        random_state=42
    )
)

search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)
search.fit(train_aligned_embs, y_train)

print("Best C (inverse reg. strength):", search.best_params_["logisticregression__C"])
print("CV ROC AUC:", search.best_score_)


best_clf = search.best_estimator_
y_pred_probs = best_clf.predict_proba(test_aligned_embs)[:, 1]
y_pred       = best_clf.predict(test_aligned_embs)

print(classification_report(y_test, y_pred, digits=4))
print("Test ROC AUC:", roc_auc_score(y_test, y_pred_probs))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best C (inverse reg. strength): 0.005
CV ROC AUC: 0.6590355375682642
              precision    recall  f1-score   support

           0     0.3359    0.1727    0.2281       249
           1     0.8171    0.9154    0.8634      1005

    accuracy                         0.7679      1254
   macro avg     0.5765    0.5441    0.5458      1254
weighted avg     0.7215    0.7679    0.7373      1254

Test ROC AUC: 0.6373154308777398
